# Can hitters change their behavior in different game situations? #
In recent years, there has a proliferation of defensive shifts in baseball. Teams identify a hitter's tendencies, and then change their positioning on the field to maximize their chances of getting the hitter out. We've seen this in the most extreme where the right side of the infield is overloaded - the 3rd baseman plays at the shortstop position, the shortstop is positioned behind 2nd, and the 2nd baseman is in short right field.

The obvious question for a fan seeing a defensive shift is, why don't hitters just hit it to the 3rd base side and take the single? Instead, hitters often hit the ball right into the shift. As long as the shift is working, teams will continue to use it to increase their chances of getting hitters out.

There is another question that arises about hitters - can hitters change their natural tendencies to match the game situation? For example, can hitters who don't hit a lot of fly balls hit a fly ball with a runner on 3rd and no outs to get the sacrifice? Can natural fly ball hitters hit a ground ball to the right side with a runner on 2nd and no outs? There are scenarios when a hitters behavior could have a benefit for the team's ability to score a run.

There is current trend of applying a defensive shift, primarily for left-handed pull hitters. The result is a defensive alignment where the 3rd baseman plays at the shortstop position, the shortstop is behind 2nd base in short center field, and the 2nd baseman is shifted over toward 1st base in short right field. The obvious question is, why don't hitters just adjust their tendencies and hit the ball somewhere else?

## Situational performance scenarios ##
For any game situation, are some players better than other at adjusting their behavior to match the situation? Alternatively, we could ask if there is a change in behavior, whether intentional or not.

* Rockies fans watched for years as Troy Tulowitski came to bat with runners on base and hit a long fly ball out. It's hard not to wonder if he's trying to hit a home run. Whether intentional or unintentional, can we determine if his behavior changes with runners on base, especially in late innings with the game on the line? Is he trying to be the hero?

* With a runner on third and less than two outs, it's nice to see that run score. Yet, batters aren't always successful in hitting a fly ball in that situation. Can we determine if some batters are more likely to hit a fly ball in that situation than they would normally?

* With a runner on 2nd and no outs, getting the runner to 3rd means that the runner scores on a sacrifice fly. In a close game, moving this runner could be key to scoring a winning or tying run. Can we determine if some batters can hit a right-side ground ball in this situation even if they don't normally hit ground balls to the right side?



### Challenges ###
* Using standard statistical methods requires knowing the underlying distribution.
* Distribution might not be normal
* Different sample sizes in different situations skew the results.

# Bootstrapping #
We're going to use a technique called bootstrapping to answer questions about players' behavior in different situations. We could also use this approach to determine run production at Coors Field is an outlier compared to other ballparks. 

* Bootstrapping is a technique where we simulate the outcome of a process by sampling randomly from a data set. 

* We use these samples to generate a distribution. 

* We don't need to know the parameters of the distribution, such as mean, std, and median ahead of time, we generate them from our sampling.

* Works with small datasets.

* Why is it called Bootstrapping? There's an expression, "Pull yourself up by your bootstraps." In statistics, it's accomplishing what you need with what you've got without sacrificing statistical rigor.

## A non-baseball example ##
Let's say you're sitting outside the engineering center one day and you see 10 people walk by, wearing different color jackets. You see,

* 5 black jackets
* 2 red jackets
* 1 blue jacket
* 2 green jacket

From these observations, you wonder what the probability is of seeing a student at CU wearing a red jacket. Your first thought is that it's 20\%. But, your sample size is so small and there are 30K students at CU. What are the mean, std of your observations? 

The information that you have is the empirical distribution of the data, not the underlying distribution. 

We can use bootstrapping and approximate the underlying distribution from these 10 observations by simulating the process of observing 10 other people, many times. With enough samples, the empirical distribution matches the underlying distribution.

We randomly draw from the possible values uniformly. In this example, $n = 10$, we can create an array of the 4 distinct values observed in the $n$ observations:
$$[B, B, B, B, B, R, R, BL, G, G]$$

__Bootstrap Resample:__ A set of $n$ draws from the original sample with replacement.

__What does sample with replacement mean?__

__Draw:__ A draw is a process of selecting one element randomly from our original sample.

### Bootstrap Resample ###
To generate a bootstrap resample in this example, we generate a random number, 1-10, that represents a position in our array of observations. For example, if we randomly generate the numbers 3 and 6, that means that we observed a B and R jacket because they are in the 3 and 6 positions in our array. We need $n$ draws for a complete resample. 

Assume we generate the following draws:

$$[4, 6, 7, 3, 6, 5, 10, 1, 2, 9]$$

it is equivalent to observing:

$$[B, R, R, B, R, B, G, B, B, G]$$

Three of the observations are R, which is equivalent to observing 3/10 red jackets. To generate a distribution of observations, we repeat the resampling process many times, let's say 1,000. We have 1,000 iterations of $n$ draws, and in each of the $n$ draws, there is a number of times that B is observed.

We can implement this process in Python pretty easily. In one run of 1000 resamples, I produced the following values:

* 0-118
* 1-275
* 2-309
* 3-192
* 4-74
* 5-19
* 6-7
* 7-1
* 8-0
* 9-0
* 10-0

The descriptive statistics for this distribution include:

* count=1000.000000
* mean=2.066000
* std=1.308953
* min=0.000000
* 25\%=1.000000
* 50\%=2.000000
* 75\%=3.000000
* max=7.000000


In [1]:
from random import randint
import pandas as pd 
#array of jacket values
jackets = [0,0,1,2,3]
trials = [0] * 1000
for i in range(0,1000): #1000 trials
    countForTrial = 0
    for j in range(0,10): #10 jackets observed, get their color
        x = randint(0,9)
        if x == 5 or x == 6:
            countForTrial += 1

    trials[i] = countForTrial

trials.sort()
df = pd.DataFrame(trials)
print(df.describe())

#What do the values mean here?

                 0
count  1000.000000
mean      2.084000
std       1.304855
min       0.000000
25%       1.000000
50%       2.000000
75%       3.000000
max       7.000000


## A Baseball Example ##
Tulo...Tulo...Tulo. Did Troy Tulowitski hit more flyballs with a runner on third and less than two outs than he did in other base-out situations, with less than two outs? 

We can't know a player's intent. But, one would hope that getting that run to score from third would be a priority for the batter, or at least that's what old-school baseball fans would say.

From the data, can we determine if there was a difference in the distribution of flyballs for Tulo in this situation?

### What data do we need? ###
* The number of times that Tulo batted with a runner on third and less than two outs, and how many times that resulted in a fly ball.
* The number of times that Tulo batted with no runner on third and less than two outs, and how many times that resulted in a fly ball.

#### Data results ####

* 143 times tulo batted with a runner on third and less than 2 outs
  - 16 times tulo hit a fly-ball out to the outfield

* 1087 times tulo batted with no runner on third and less than 2 outs
  - 132 times tulo hit a fly-ball out to the outfield with no runner on third

__Null Hypothesis: There is no change in Tulo's behavior with a runner on third when there is less than 2 outs.__

* The pool of possible values is 143+1087=1230 at-bats. 
  - 16 of 143 is flyball and 132 of 1087 is flyball.

* A draw is an at-bat. The result of the draw is he either hits a flyball or he doesn't.  

* A sample is 143 at-bats for Tulo to simulate the times he came to bat with runner on third and less than 2 outs.

#### Setting up the simulation ####
* In all of his at-bats, how many times did Tulo hit a flyball out? 148

* Need a list of 1230 at-bats, where 0-147 = 1 for flyball out and 148-1229 = 0 for not flyball out

### Simulation results ###
In one run of the simulation, 16 of 143 shows up in 96 out of 1000 trials, so p-value of observing 16 flyballs in 143 at bats is .096. Other runs of the simulation produced similar results. We can't reject the null hypothesis. There is no change in his flyball hitting behavior with a runner on third. 


In [48]:
from random import randint
import pandas as pd 
import matplotlib.pylab as plt

#tulo and the flyball out
hits = [0] * 1230
for i in range(0,148):
    hits[i] = 1
trials = [0] * 1000
for i in range(0,1000):
    countForTrial = 0
    for j in range(0,143):
        x = randint(0,1229)
        countForTrial+=hits[x]
    trials[i] = countForTrial
trials.sort()
tr = 0
for t in trials:
    if t == 16:
        tr+=1
print(tr/1000)
df = pd.DataFrame(trials)
print(df.describe())
# Initialize figure and axis
fig, ax = plt.subplots(figsize=(8,4))

# Plot histogram 
df.hist(ax=ax);
# Add a title
ax.set_title("Tulo flyball outs", fontsize=20)

# Add axis labels 
ax.set_xlabel("Number of flyballs in 143 at-bats", fontsize=16)
ax.set_ylabel("Frequency", fontsize=16)

0.072


Text(0,0.5,'Frequency')

## Questions ##
1. (2 pts) Explain how you could use bootstrapping to determine how run scoring at Coors Field compares to the rest of the National League? Specifically, how does the mean number of runs scored at Coors in the 2018 season compare to the mean number of runs scored at other stadiums?
   
   1a. What is a draw for this problem?
   
   1b. What is a sample?
   
   1c. How do you set up your simulation?
   
   1d. How would you determine if run scoring at Coors is a statistically significant outlier?
   
   1e. What is the Null hypothesis?
   
2. (2 pts) Choose your favorite player who had at least 500 plate appearances in 2018. How would you determine if he had a statistically significant percentage of strike outs with runners on second and third and no outs than he did with bases empty and no outs?     
   
   2a. What is a draw for this problem?
   
   2b. What is a sample?
   
   2c. How do you set up your simulation?
   
   2d. What is the Null hypothesis?

3. (2 pts) Choose either problem 1 or 2 and implement it and explain the result that you get.

In [2]:
from pybaseball import statcast
data = statcast(start_dt='2018-04-07', end_dt='2018-11-08',team = 'COL')
data.head(10)


This is a large query, it may take a moment to complete
Completed sub-query from 2018-04-07 to 2018-04-12
Completed sub-query from 2018-04-13 to 2018-04-18
Completed sub-query from 2018-04-19 to 2018-04-24
Completed sub-query from 2018-04-25 to 2018-04-30
Completed sub-query from 2018-05-01 to 2018-05-06
Completed sub-query from 2018-05-07 to 2018-05-12
Completed sub-query from 2018-05-13 to 2018-05-18
Completed sub-query from 2018-05-19 to 2018-05-24
Completed sub-query from 2018-05-25 to 2018-05-30
Completed sub-query from 2018-05-31 to 2018-06-05
Completed sub-query from 2018-06-06 to 2018-06-11
Completed sub-query from 2018-06-12 to 2018-06-17
Completed sub-query from 2018-06-18 to 2018-06-23
Completed sub-query from 2018-06-24 to 2018-06-29
Completed sub-query from 2018-06-30 to 2018-07-05
Completed sub-query from 2018-07-06 to 2018-07-11
Completed sub-query from 2018-07-12 to 2018-07-17
Completed sub-query from 2018-07-18 to 2018-07-23
Completed sub-query from 2018-07-24 to 2018-

,index,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,...,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment
0,1295,FF,2018-10-07,96.2,3.4110,5.0616,Josh Hader,435622.0,623352.0,field_out,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Strategic,Standard
1,1297,FF,2018-10-07,98.1,3.0280,5.1418,Josh Hader,435622.0,623352.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Strategic,Standard
2,1299,FF,2018-10-07,97.2,3.2318,5.0824,Josh Hader,471865.0,623352.0,strikeout,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Strategic,Standard
3,1301,FF,2018-10-07,99.1,2.9120,5.1710,Josh Hader,471865.0,623352.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Strategic,Standard
4,1303,FF,2018-10-07,97.1,3.1104,5.1852,Josh Hader,471865.0,623352.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Strategic,Standard
5,1305,FF,2018-10-07,96.3,3.1577,5.0183,Josh Hader,471865.0,623352.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Standard,Standard
6,1307,FF,2018-10-07,96.8,2.9912,5.1534,Josh Hader,471865.0,623352.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Standard,Standard
7,1308,FT,2018-10-07,95.7,-0.9320,5.6497,Jeremy Jeffress,407812.0,502026.0,walk,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Infield shift,Standard
8,1310,FF,2018-10-07,95.0,-0.9156,5.7597,Jeremy Jeffress,407812.0,502026.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Infield shift,Standard
9,1312,KC,2018-10-07,81.3,-1.1819,5.6720,Jeremy Jeffress,407812.0,502026.0,NaN,...,0.0,6.0,0.0,6.0,6.0,0.0,0.0,6.0,Infield shift,Standard


In [47]:
import numpy as np
df = data[['index',
 'batter',
 'home_team',
 'away_team',
 'balls',
 'strikes',
 'on_3b',
 'on_2b',
 'on_1b',
 'outs_when_up',
 'at_bat_number',
          'inning_topbot',
          'events']]
df = df.loc[(df['batter']==571448)]

df = df.dropna(subset=['events'])

df.head(10)

df2 = df
df2 = df2.fillna(0)
df1 = df.loc[(df['on_2b'] >=0) & (df['on_3b'] >=0) & (df['outs_when_up'] == 2)]
df2 = df2.loc[(df2['on_2b']  == 0) & (df2['on_3b'] == 0) & (df2['outs_when_up'] == 0)& (df2['on_1b'] == 0)]


df2.head(1000)

so = df1.query('events == "strikeout"').events.count()
so2 = df2.query('events == "strikeout"').events.count()
print("For the first situation there was 2 SO out of 5 attempts")
print("For the first situation there was",so2,"out of 131 attempts")
start=[]
for i in range(5000):
    count = 0
    for j in range(5):
        x = randint(1,5)
        if (x  <=  2):
            count = count +1
    start.append(count/5)

nexts=[]
for i in range(5000):
    count = 0
    for j in range(131):
        x = randint(1,131)
        if (x  <=  24):
            count = count +1
    nexts.append(count/131)

            
print(np.mean(start))
print(np.mean(nexts))

For the first situation there was 2 SO out of 5 attempts
For the first situation there was 24 out of 131 attempts
0.39952000000000004
0.1833053435114504


For the first situation the Strikeout percent was .399 the strikeout percentage for the second situation was .183. As the strikeout percentage is almost double you can clear see there is a diffrence in strike out percentage in these two simulations, but sadly I don't think this should be accept as Arednado of his almost 600 at bats during 2018 only came up to the first situations 5 times. So really it would be nice to have more data. Also important to note he did hit a home run in one of the 5 at bats. 